In [28]:
import nltk
from nltk import word_tokenize, sent_tokenize, pos_tag
import pandas as pd
import os
import re
from xml.dom import minidom
from xml.etree import cElementTree as ET

In [29]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\24526\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\24526\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

Data Loading

In [30]:
# Load data from local/url

In [31]:
path = r'.\data\train_xml\ADREVIEW.xml'

text_content = ''

tree = ET.parse(path)
root = tree.getroot()

for child in root:
    print(child.tag)


Text
Mentions
Relations
Reactions


Data Preprocessing

In [32]:
def spans(txt):
    tokens=nltk.word_tokenize(txt)
    offset = 0
    for token in tokens:
        offset = txt.find(token, offset)
        yield token, offset, offset+len(token)
        offset += len(token)

In [33]:
def BIO_tagging(tags, text):
    """
    This function tags the text with BIO tags
    
    Input: An element of a list of tags, extracted text content
    Output: A set of tuples which consist of words and BIO tag
    """
    
    tagged_words = []
    tagged_position = []
    
    tokens = spans(text)
    
    for token in tokens:

        tagged = False
    
        for i in range(len(tags)):

            pos = tags[i].attrib['start'].split(',')
            length = tags[i].attrib['len'].split(',')

            if len(pos) == 1:
                
                d_pos = int(pos[0])
                d_length = int(length[0])

                if token[1] != d_pos and (d_pos >= token[1] or (d_pos+d_length) <= token[1]):
                    continue      
                else:                  
                    if token[1] == d_pos :
                        B_tag = (token[0],'B-'+ str(tags[i].attrib['type']))
                        tagged_words.append(B_tag)
                        tagged = True
                       
                        
                    elif d_pos < token[1] < d_pos+d_length:
                        I_tag = (token[0],'I-'+ str(tags[i].attrib['type']))
                        tagged_words.append(I_tag)
                        tagged = True
                        
            else:

                for m in range(len(pos)):

                    c_pos = int(pos[m])
                    c_length = int(length[m])

                    if token[1] != c_pos and (c_pos >= token[1] or (c_pos+c_length) <= token[1]):
                        continue            
                    else:
                        
                        if token[1] == c_pos and m == 0:
                            B_tag = (token[0],'B-'+ str(tags[i].attrib['type']))
                            tagged_words.append(B_tag)
                            tagged = True
                            break
                        
                        elif token[1] == c_pos and m != 0:
                            I_tag = (token[0],'I-'+ str(tags[i].attrib['type']))
                            tagged_words.append(I_tag)
                            tagged = True
                            break
                            
                        elif c_pos < token[1] < c_pos+c_length:
                            I_tag = (token[0],'I-'+ str(tags[i].attrib['type']))
                            tagged_words.append(I_tag)
                            tagged = True
                            break
                        
            if tagged == True:
                break
    
        if tagged == False:
            O_tag = (token[0],'O')
            tagged_words.append(O_tag)

    return tagged_words
        
        

In [34]:
def  preprocess(string):
    """
    Preprocessing for all datasets
    
    Input: A sentence from original text
    Output: Preprocessed sentence
    """
    
    string = re.sub(r"[^\w(),|!?\'\`\:\-\.;\$%#]", " ", string)
    string = re.sub(r"\'s", " is", string)
    string = re.sub(r"\'ve", " have", string)
    string = re.sub(r"n\'t", " not", string)
    string = re.sub(r"\'re", " are", string)
    string = re.sub(r"\'d", " would", string)
    string = re.sub(r"\'ll", " will", string)
    string = re.sub(r"(?<=\w)\.\.\.", " ... ", string)
    string = re.sub(r"(?<=\w)\.", " . ", string)
    string = re.sub(r"(?<=\w),", " , ", string)
    string = re.sub(r"(?<=\w);", " ; ", string)
    string = re.sub(r"(?<=\w)!", " ! ", string)
    string = re.sub(r"\((?=\w)", " ( ", string)
    string = re.sub(r"(?<=\w)\)", " ) ", string)
    string = re.sub(r"(?<=\w)\?", " ? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    
    
    #Tokenization
    tokens = word_tokenize(string)

    # Part-of-speech Tagging
    pos_tagged = pos_tag(tokens)
       
    
    return pos_tagged

In [35]:
# Tagged BIO tagger for the token set

for section in root[0].findall('Section'):
    tags = []
    for mention in root[1].findall('Mention'):
        if mention.attrib['section'] == section.attrib['id']:
            tags.append(mention)
    tagged = BIO_tagging(tags, section.text)
    print(tagged)


[('6', 'O'), ('ADVERSE', 'O'), ('REACTIONS', 'O'), ('EXCERPT', 'O'), (':', 'O'), ('Serious', 'B-Severity'), ('hypersensitivity', 'B-AdverseReaction'), ('reactions', 'I-AdverseReaction'), ('have', 'O'), ('been', 'O'), ('reported', 'O'), ('following', 'O'), ('AdreView', 'O'), ('administration', 'O'), ('.', 'O'), ('The', 'O'), ('most', 'O'), ('common', 'O'), ('adverse', 'O'), ('reactions', 'O'), (',', 'O'), ('dizziness', 'B-AdverseReaction'), (',', 'O'), ('rash', 'B-AdverseReaction'), (',', 'O'), ('pruritis', 'B-AdverseReaction'), (',', 'O'), ('flushing', 'B-AdverseReaction'), (',', 'O'), ('headache', 'B-AdverseReaction'), (',', 'O'), ('and', 'O'), ('injection', 'B-AdverseReaction'), ('site', 'I-AdverseReaction'), ('hemorrhage', 'I-AdverseReaction'), ('occurred', 'O'), ('in', 'O'), ('<', 'O'), ('1.3', 'O'), ('%', 'O'), ('of', 'O'), ('patients', 'O'), ('.', 'O'), ('(', 'O'), ('6.1', 'O'), (',', 'O'), ('6.2', 'O'), (')', 'O'), ('To', 'O'), ('report', 'O'), ('SUSPECTED', 'O'), ('ADVERSE', 'O

In [36]:
# Tagged POS tagger for the token set
for section in root[0].findall('Section'):
    pps = preprocess(section.text)
    # print(pps)